------------------------------------------------------------------------

## Importing the libraries


In [61]:
!pip install alpha_vantage

  Using cached alpha_vantage-2.3.1-py3-none-any.whl (31 kB)



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries


## Downloading the data

In this code we have used the [yfinance](https://pypi.org/project/yfinance/) package to download the data from the [Yahoo Finance](https://finance.yahoo.com/) from the period "31-8-2021" to "31-8-2021" to "31-8-2023"


In [65]:
# Define your Alpha Vantage API key
api_key = 'J5DNDJ69XM34DC0G'
# Initialize Alpha Vantage TimeSeries
ts = TimeSeries(key=api_key, output_format='pandas')

In [63]:
start_date = "2020-01-31"
end_date = "2022-01-31"

The stocks that we have used for our problem are given below:

1.  Pharmacy - Medplus India Ltd, Dr. Reddy's Laboratories
2.  Banking - Punjab National Bank, Angel One Limited
3.  Technology - Wipro Ltd., Infosys Ltd
4.  Agriculture - Rallis India LTD, Indian Pesticides Limited
5.  Energy - Bharat Petrochemical Limited, Indian Oil Corporation


In [51]:
#| output: False
# List of stocks in consider
stocks = {                                    
    'Pharma': ['RDY', 'FORTIS.NS'],
    'Banking': ['HDB', 'SBIN.NS'],
    'Technology': ['INFY', 'TCS.NS'],
    'Agriculture': ['RALLIS.NS', 'IPL.BO'],
    'Energy': ['BPCL.NS', 'IOC.BO']
}

The code to download the data from yfinance and also to calculate the daily returns using the `Adjusted Close Price` column.


In [66]:
# Download historical price data
data = {}
for sector, symbols in stocks.items():
    sector_data = pd.DataFrame()
    for symbol in symbols:
        try:
            stock_data, _ = ts.get_daily_adjusted(symbol=symbol, outputsize='full')
            stock_data = stock_data[start_date:end_date]
            stock_data['pct_change'] = stock_data['5. adjusted close'].pct_change().dropna()
            sector_data[symbol] = stock_data['pct_change']
        except Exception as e:
            print(f"Error downloading data for {symbol}: {e}")
    data[sector] = sector_data

Error downloading data for RDY: Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints
Error downloading data for FORTIS.NS: Error getting data from the api, no return was given.
Error downloading data for HDB: Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints
Error downloading data for SBIN.NS: Error getting data from the api, no return was given.
Error downloading data for INFY: Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints
Error downloading data for TCS.NS: Error getting data from the api, no return was given.
Error downloading data for RALLIS.NS: Error getting data fr

In [53]:
#| output: true
for sector, data in sector_data.items():
    print(f"The initial values for sector {sector} are:")
    print(data.head())

The initial values for sector BPCL.NS are:
Date
2020-02-03    0.007331
2020-02-04    0.039539
2020-02-05    0.048485
2020-02-06   -0.013255
2020-02-07   -0.007373
Name: BPCL.NS, dtype: float64
The initial values for sector IOC.BO are:
Date
2020-02-03   -0.047598
2020-02-04    0.053679
2020-02-05    0.007466
2020-02-06    0.020052
2020-02-07    0.003846
Name: IOC.BO, dtype: float64


## Calculating the mean and variance of Returns


In [56]:
# Calculate mean and variance of daily returns
mu_hat = {}
v_hat = {}
for sector, sector_data in data.items():
    mu_hat[sector] = sector_data.mean()
    v_hat[sector] = sector_data.var()

AttributeError: 'float' object has no attribute 'mean'

In [9]:
#| output: true
print("Mean Return:",mu_hat)
print("Variance of Return:", v_hat)

Mean Return: {}
Variance of Return: {}


In [ ]:
# Portfolio optimization
b = 0.001  # Expected return (assumed to be constant for simplicity)
w_b = {}
for sector, sector_data in data.items():
    cov_matrix = sector_data.cov()  # Covariance matrix
    ones_vector = np.ones(len(stocks[sector]))  # Vector of ones
    inv_cov_matrix = np.linalg.inv(cov_matrix)

    w_b[sector] = np.dot(inv_cov_matrix, mu_hat[sector] - b * ones_vector) / np.sum(np.dot(inv_cov_matrix, ones_vector))

In [ ]:
# Simulate portfolio performance and calculate mean squared error
months_to_simulate = [1, 3, 6]
results = {}
for month in months_to_simulate:
    results[month] = []
    for sector, sector_data in data.items():
        try:
            port_returns = np.dot(sector_data.values, w_b[sector])  # Portfolio returns
            overall_port_returns = np.dot(port_returns, np.ones(len(port_returns)))  # Overall portfolio return
            mse = np.mean((overall_port_returns - mu_hat[sector]) ** 2)  # Mean squared error
            results[month].append(mse)
        except ValueError as e:
            print(f"Error processing {sector}: {e}")

In [ ]:
# Compare results with initial return
initial_mse = {}
for sector, sector_data in data.items():
    try:
        initial_returns = np.dot(sector_data.iloc[0], w_b[sector])  # Initial portfolio returns
        initial_mse[sector] = (initial_returns - mu_hat[sector][0]) ** 2
    except ValueError as e:
        print(f"Error processing {sector}: {e}")

In [ ]:
# Print or analyze the results as needed
print("Results for 1 month:", results[1])
print("Results for 3 months:", results[3])
print("Results for 6 months:", results[6])
print("Initial MSE:", initial_mse)